In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
impressions_df = pd.read_csv("../datasets/impressions/2023/impressions_OBOS_04_05.csv")
impressions_df

,id,hostname,beacon_ids,first_event_at_iso,ad_width_chosen,ad_height_chosen,placement_ids,placement_ids_chosen,total_fixation_duration,channel,beacon_type_chosen,ad_technical_format,is_fixated,gaze_valid,is_iab_inview,exist_viewable_1_s_threshold_50,exist_viewable_2_s_threshold_50,part_month,part_year
0,ef62d92e-ab28-47f6-8796-6379f986470b,youtube.com,"[""49640e6d-a085-400b-869c-51e21566d2ec"",""7f6d3...",2023-05-24T09:33:56.864Z,180,150,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-home-feed-v1,display,False,False,False,False,False,5,2023
1,6f7b91fd-da44-4582-b8a9-d7627335d903,youtube.com,"[""4921720c-4451-4d0e-b1af-f1f87da91fb5"",""6283c...",2023-05-24T09:33:56.864Z,180,150,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-home-feed-v1,display,False,False,False,False,False,5,2023
2,006bdc82-5068-470e-bac6-85a5f5a6815f,youtube.com,"[""192213a7-d846-4e70-9cd3-6c0226c7c03a"",""7eb19...",2023-05-24T09:33:56.864Z,180,150,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-home-feed-v1,display,False,False,False,False,False,5,2023
3,858ff1db-a03d-4307-a3e1-05696a8a2955,youtube.com,"[""000170f6-4984-4ac9-a958-d4f59c3d2d3e"",""5738c...",2023-05-24T09:34:01.196Z,180,150,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-watch-sidebar-featured-v1,display,False,False,False,False,False,5,2023
4,605f13b8-cc44-41d1-b17d-0d01283f1be8,youtube.com,"[""a5b1a310-fd49-4014-b53c-d9eaf3735bc9""]",2023-05-24T09:34:01.197Z,180,150,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-home-feed-v1,display,False,False,False,False,False,5,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4240350,b9f29f1f-3ed9-46b1-8249-455ff095aef6,jigsawplanet.com,"[""7d90cacb-6f87-43a3-a0c3-1ff3a7ab8526"",""80354...",2023-04-25T20:44:20.233Z,980,120,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/doubleclick-img-link-google-v1,display,False,True,True,True,True,4,2023
4240351,c7b35ba8-195c-4ce4-a327-cf0ae4288a7d,youtube.com,"[""4a8a302e-94bc-4d44-834b-c5db7f19fa2a""]",2023-04-25T20:54:10.235Z,468,240,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-watch-sidebar-featured-v1,display,False,True,True,True,True,4,2023
4240352,8cc32420-cadc-4e1c-817b-5d7d6f6613ef,youtube.com,"[""810d4796-a5b4-4386-8ea8-3822cbc14d51""]",2023-04-25T20:54:10.235Z,300,250,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-watch-sidebar-featured-v1,display,False,True,False,False,False,4,2023
4240353,72872aa3-c1d8-41da-b34b-e282f9c8f931,youtube.com,"[""11ef0c0a-0593-4689-8072-62fdc378e42c"",""3f1fc...",2023-04-25T20:54:10.235Z,468,120,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,155,display,ad/youtube.com-watch-companion-v1,display,True,True,True,True,True,4,2023


In [3]:
import json

def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    ret = ''
    try:
        ret = placement_id['tag_id'][0]
    except:
        ret = ','.join([bas for bas in final_bam_ad_slots])
    else:
        ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret
    

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)
impressions_df = impressions_df[impressions_df['pid_type'] == 'int']
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))

/tmp/ipykernel_29031/2872913832.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))


In [4]:
# run through predictions
import os
import json


def _get_pred_map():
    pred_map = dict()
    
    for i in range(1, 5):
        if i < 10:
            i = f'0{i}'
        for day in os.listdir(f'../predictions/2023/{i}'):
            for h in os.listdir(f'../predictions/2023/{i}/{day}'):
                if not h.endswith('.ndjson'):
                    continue
                with open(f'../predictions/2023/{i}/{day}/{h}')as f:
                    for line in f.readlines():
                        json_line = json.loads(line)
                        id_ = json_line['id']
                        pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [5]:
def _get_final_fixation(gaze_valid, is_fixated, id_, pred_map):
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:
        return pred_map[id_]
    return False

impressions_df['final_fixation'] = impressions_df.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)

/tmp/ipykernel_29031/4199445804.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impressions_df['final_fixation'] = impressions_df.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)


In [6]:
impressions_df["true_is_iab_inview"] = impressions_df.apply(
    lambda x: 
    True if (
        (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
    ) | (x["is_iab_inview"] == True) 
    else False, 
    axis=1
)

/tmp/ipykernel_29031/293613858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impressions_df["true_is_iab_inview"] = impressions_df.apply(


In [7]:
grouped_df = impressions_df.groupby(["pid", "hostname"]).apply(
    lambda x: pd.Series(
        {
            "num_impressions": x["id"].count(),
            "num_fixations": x.loc[x["final_fixation"], "id"].count(),
            "is_iab_inview": x.loc[x["true_is_iab_inview"], "id"].count(),
        }
    )
)
grouped_df = grouped_df.reset_index()

In [8]:
grouped_df["fixation_ratio"] = round(grouped_df["num_fixations"] / grouped_df["num_impressions"] * 100, 2)
grouped_df["inview_ratio"] = round(grouped_df["is_iab_inview"] / grouped_df["num_impressions"] * 100, 2)
grouped_df

,pid,hostname,num_impressions,num_fixations,is_iab_inview,fixation_ratio,inview_ratio
0,7340711,tripadvisor.co.uk,2,0,0,0.00,0.00
1,9152456,dn.se,2,0,2,0.00,100.00
2,10266284,illvet.se,1,1,1,100.00,100.00
3,10266412,myabandonware.com,1,1,0,100.00,0.00
4,10266412,tripstodiscover.com,2,0,0,0.00,0.00
...,...,...,...,...,...,...,...
2412,97893164,dailymail.co.uk,3,0,0,0.00,0.00
2413,97893164,ebay.co.uk,59,1,4,1.69,6.78
2414,97893164,forums.guru3d.com,1,0,0,0.00,0.00
2415,97893164,grimsbytelegraph.co.uk,1,0,0,0.00,0.00


In [9]:
obos_impressions_control_group = pd.read_excel("../datasets/xaxis_attention_report_obos_control.xlsx")
obos_control_group_pids = list(obos_impressions_control_group["Placement Id"].unique())
obos_impressions_test_group = pd.read_excel("../datasets/xaxis_attention_report_obos_test.xlsx")
obos_test_group_pids = list(obos_impressions_test_group["Placement Id"].unique())


In [10]:
# Filter impressions based on group
group = "test"
filtered_impressions = grouped_df[grouped_df["pid"].isin(obos_test_group_pids)]
filtered_impressions

,pid,hostname,num_impressions,num_fixations,is_iab_inview,fixation_ratio,inview_ratio
1,9152456,dn.se,2,0,2,0.0,100.00
2,10266284,illvet.se,1,1,1,100.0,100.00
5,10266423,hitta.se,8,2,5,25.0,62.50
33,11329201,nettavisen.no,2,0,0,0.0,0.00
50,11576944,hitta.se,1,0,1,0.0,100.00
...,...,...,...,...,...,...,...
1907,25659495,verymuchrussian.com,1,0,0,0.0,0.00
1908,25659495,w3schools.com,7,0,5,0.0,71.43
1909,25659495,wrestlinginc.com,7,0,5,0.0,71.43
1910,25659495,zelda.fandom.com,1,0,0,0.0,0.00


In [11]:
merged_df = pd.merge(filtered_impressions, obos_impressions_test_group, left_on=["pid"], right_on=["Placement Id"], how="outer")
merged_df

,pid,hostname,num_impressions,num_fixations,is_iab_inview,fixation_ratio,inview_ratio,Date,Hour,LI Id,LI Name,Placement Id,Size,domain,Imps,Viewability Measurement Rate %,Viewability Rate %,Custom Viewability Rate %
0,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653838,Linköping AS10 #462965,9152456,250x600,dn.se,3,1.0,0.666667,0.666667
1,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653835,Norrköping AS10 #462965,9152456,980x240,dn.se,2,1.0,1.000000,1.000000
2,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20632313,Stockholm - Nacka Barkarby Täby Haninge AS10 #...,9152456,980x240,dn.se,2,1.0,1.000000,0.500000
3,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653915,Helsingborg AS10 #462965,9152456,980x240,dn.se,2,1.0,0.500000,0.500000
4,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653909,Eskilstuna AS10 #462965,9152456,640x320,dn.se,2,1.0,0.500000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-06,17,20632308,Sthlm Nacka AS10 #462965,21796701,320x320,innebandymagazinet.se,1,1.0,1.000000,0.000000
769329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-10,11,20652583,Sthlm Haninge AS10 #462965,21796701,320x320,innebandymagazinet.se,1,1.0,1.000000,1.000000
769330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-12,21,20633767,Sthlm Barkarby AS10 #462965,21796701,320x320,innebandymagazinet.se,1,1.0,0.000000,0.000000
769331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09,6,20653916,Halmstad AS10 #462965,19256727,320x320,blt.se,1,1.0,0.000000,0.000000


In [12]:
merged_df = merged_df.dropna(subset=["pid"])
merged_df = merged_df.dropna(subset=["Imps"])
merged_df

,pid,hostname,num_impressions,num_fixations,is_iab_inview,fixation_ratio,inview_ratio,Date,Hour,LI Id,LI Name,Placement Id,Size,domain,Imps,Viewability Measurement Rate %,Viewability Rate %,Custom Viewability Rate %
0,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653838,Linköping AS10 #462965,9152456,250x600,dn.se,3,1.0,0.666667,0.666667
1,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653835,Norrköping AS10 #462965,9152456,980x240,dn.se,2,1.0,1.000000,1.000000
2,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20632313,Stockholm - Nacka Barkarby Täby Haninge AS10 #...,9152456,980x240,dn.se,2,1.0,1.000000,0.500000
3,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653915,Helsingborg AS10 #462965,9152456,980x240,dn.se,2,1.0,0.500000,0.500000
4,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653909,Eskilstuna AS10 #462965,9152456,640x320,dn.se,2,1.0,0.500000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344900,27264940.0,svd.se,4.0,0.0,0.0,0.0,0.0,2023-05-11,7,20653910,Uppsala AS10 #462965,27264940,320x400,svd.se,1,1.0,0.000000,0.000000
344901,27264940.0,svd.se,4.0,0.0,0.0,0.0,0.0,2023-05-11,17,20653918,Karlstad AS10 #462965,27264940,320x320,svd.se,1,1.0,0.000000,0.000000
344902,27264940.0,svd.se,4.0,0.0,0.0,0.0,0.0,2023-05-11,21,20633767,Sthlm Barkarby AS10 #462965,27264940,320x320,svd.se,1,1.0,1.000000,1.000000
344903,27264940.0,svd.se,4.0,0.0,0.0,0.0,0.0,2023-05-11,21,20633767,Sthlm Barkarby AS10 #462965,27264940,320x400,svd.se,1,1.0,1.000000,1.000000


In [13]:
merged_df[merged_df["Imps"].isnull()]

,pid,hostname,num_impressions,num_fixations,is_iab_inview,fixation_ratio,inview_ratio,Date,Hour,LI Id,LI Name,Placement Id,Size,domain,Imps,Viewability Measurement Rate %,Viewability Rate %,Custom Viewability Rate %


In [14]:
merged_df["client_based_is_iab_inview"] = merged_df.apply(lambda x: math.floor(x["Imps"] * (x["inview_ratio"] / 100)), axis=1)
merged_df["client_based_inview_ratio"] = round(merged_df["client_based_is_iab_inview"] / merged_df["Imps"] * 100, 2)
merged_df["client_based_fixations"] = merged_df.apply(lambda x: math.floor(x["Imps"] * (x["fixation_ratio"] / 100)), axis=1)
merged_df["client_fixation_rate"] = round(merged_df["client_based_fixations"] / merged_df["Imps"] * 100, 2)
merged_df

,pid,hostname,num_impressions,num_fixations,is_iab_inview,fixation_ratio,inview_ratio,Date,Hour,LI Id,...,Size,domain,Imps,Viewability Measurement Rate %,Viewability Rate %,Custom Viewability Rate %,client_based_is_iab_inview,client_based_inview_ratio,client_based_fixations,client_fixation_rate
0,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653838,...,250x600,dn.se,3,1.0,0.666667,0.666667,3,100.0,0,0.0
1,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653835,...,980x240,dn.se,2,1.0,1.000000,1.000000,2,100.0,0,0.0
2,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20632313,...,980x240,dn.se,2,1.0,1.000000,0.500000,2,100.0,0,0.0
3,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653915,...,980x240,dn.se,2,1.0,0.500000,0.500000,2,100.0,0,0.0
4,9152456.0,dn.se,2.0,0.0,2.0,0.0,100.0,2023-04-18,17,20653909,...,640x320,dn.se,2,1.0,0.500000,0.000000,2,100.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344900,27264940.0,svd.se,4.0,0.0,0.0,0.0,0.0,2023-05-11,7,20653910,...,320x400,svd.se,1,1.0,0.000000,0.000000,0,0.0,0,0.0
344901,27264940.0,svd.se,4.0,0.0,0.0,0.0,0.0,2023-05-11,17,20653918,...,320x320,svd.se,1,1.0,0.000000,0.000000,0,0.0,0,0.0
344902,27264940.0,svd.se,4.0,0.0,0.0,0.0,0.0,2023-05-11,21,20633767,...,320x320,svd.se,1,1.0,1.000000,1.000000,0,0.0,0,0.0
344903,27264940.0,svd.se,4.0,0.0,0.0,0.0,0.0,2023-05-11,21,20633767,...,320x400,svd.se,1,1.0,1.000000,1.000000,0,0.0,0,0.0


In [15]:
merged_df["num_fixations"] = merged_df["num_fixations"].astype("Int64")
merged_df["num_impressions"] = merged_df["num_impressions"].astype("Int64")

In [16]:
def get_sample_size(num_impressions):
    if np.isnan(num_impressions):
        return "N/A"
    if num_impressions <= 20:
        return "low"
    elif num_impressions > 20 and num_impressions <= 100:
        return "medium"
    else:
        return "high"
    
merged_df['sample_size'] = merged_df['num_impressions'].apply(lambda num_impressions: get_sample_size(num_impressions))
merged_df

,pid,hostname,num_impressions,num_fixations,is_iab_inview,fixation_ratio,inview_ratio,Date,Hour,LI Id,...,domain,Imps,Viewability Measurement Rate %,Viewability Rate %,Custom Viewability Rate %,client_based_is_iab_inview,client_based_inview_ratio,client_based_fixations,client_fixation_rate,sample_size
0,9152456.0,dn.se,2,0,2.0,0.0,100.0,2023-04-18,17,20653838,...,dn.se,3,1.0,0.666667,0.666667,3,100.0,0,0.0,low
1,9152456.0,dn.se,2,0,2.0,0.0,100.0,2023-04-18,17,20653835,...,dn.se,2,1.0,1.000000,1.000000,2,100.0,0,0.0,low
2,9152456.0,dn.se,2,0,2.0,0.0,100.0,2023-04-18,17,20632313,...,dn.se,2,1.0,1.000000,0.500000,2,100.0,0,0.0,low
3,9152456.0,dn.se,2,0,2.0,0.0,100.0,2023-04-18,17,20653915,...,dn.se,2,1.0,0.500000,0.500000,2,100.0,0,0.0,low
4,9152456.0,dn.se,2,0,2.0,0.0,100.0,2023-04-18,17,20653909,...,dn.se,2,1.0,0.500000,0.000000,2,100.0,0,0.0,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344900,27264940.0,svd.se,4,0,0.0,0.0,0.0,2023-05-11,7,20653910,...,svd.se,1,1.0,0.000000,0.000000,0,0.0,0,0.0,low
344901,27264940.0,svd.se,4,0,0.0,0.0,0.0,2023-05-11,17,20653918,...,svd.se,1,1.0,0.000000,0.000000,0,0.0,0,0.0,low
344902,27264940.0,svd.se,4,0,0.0,0.0,0.0,2023-05-11,21,20633767,...,svd.se,1,1.0,1.000000,1.000000,0,0.0,0,0.0,low
344903,27264940.0,svd.se,4,0,0.0,0.0,0.0,2023-05-11,21,20633767,...,svd.se,1,1.0,1.000000,1.000000,0,0.0,0,0.0,low


In [17]:
result_df = merged_df[["hostname", "Placement Id", "Imps", "client_based_fixations", "client_based_is_iab_inview", "num_impressions", "num_fixations", "is_iab_inview"]]
result_df_grouped = result_df.groupby(["hostname"]).apply(
    lambda x: pd.Series(
        {
            "client_imps": x["Imps"].sum(),
            "num_imps": x["num_impressions"].sum(),
            "client_fixations": x["client_based_fixations"].sum(),
            "num_fixations": x["num_fixations"].sum(),
            "client_num_is_iab_inview": x["client_based_is_iab_inview"].sum(),
            "num_is_iab_inview": x["is_iab_inview"].sum()
        }
    )
)
result_df_grouped = result_df_grouped.reset_index()
result_df_grouped["sample_size"] = result_df_grouped["num_imps"].apply(lambda x: get_sample_size(x))
result_df_grouped["Fixation Ratio"] = round(result_df_grouped["client_fixations"] / result_df_grouped["client_imps"] * 100, 2)
result_df_grouped["Inview Ratio"] = round(result_df_grouped["client_num_is_iab_inview"] / result_df_grouped["client_imps"] * 100, 2)
result_df_grouped["Inview / Fixation Ratio"] = round(result_df_grouped["client_fixations"] / result_df_grouped["client_num_is_iab_inview"] * 100, 2)
result_df_grouped

,hostname,client_imps,num_imps,client_fixations,num_fixations,client_num_is_iab_inview,num_is_iab_inview,sample_size,Fixation Ratio,Inview Ratio,Inview / Fixation Ratio
0,affarsvarlden.se,2800.0,6060.0,0.0,0.0,1168.0,4545.0,high,0.00,41.71,0.00
1,aftonbladet.se,965536.0,234907422.0,235794.0,72749870.0,322212.0,129420376.0,high,24.42,33.37,73.18
2,allabolag.se,18667.0,15387.0,0.0,0.0,3327.0,5129.0,high,0.00,17.82,0.00
3,archive.md,2400.0,516.0,0.0,0.0,2400.0,516.0,high,0.00,100.00,0.00
4,azaleasdolls.com,1.0,3.0,0.0,0.0,1.0,3.0,low,0.00,100.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
75,wowhead.com,487.0,2124.0,0.0,0.0,7.0,708.0,high,0.00,1.44,0.00
76,wrestlezone.com,443.0,419.0,0.0,0.0,0.0,0.0,high,0.00,0.00,NaN
77,wrestlinginc.com,2783.0,23566.0,0.0,1188.0,2484.0,20000.0,high,0.00,89.26,0.00
78,zelda.fandom.com,1.0,1.0,0.0,0.0,0.0,0.0,low,0.00,0.00,NaN


In [18]:
final_df = result_df_grouped[["hostname", "client_imps", "client_fixations", "Fixation Ratio", "client_num_is_iab_inview", "Inview Ratio", "Inview / Fixation Ratio", "sample_size"]]
final_df = final_df.rename(columns={"client_imps": "Impressions", "client_fixations": "Fixations", "client_num_is_iab_inview": "Inview", "sample_size": "Sample Size"})
final_df["Inview / Fixation Ratio"] = final_df["Inview / Fixation Ratio"].fillna(0)
final_df["Inview / Fixation Ratio"] = final_df["Inview / Fixation Ratio"].replace([np.inf, -np.inf], 0)
final_df["Impressions"] = final_df["Impressions"].astype("Int64")
final_df["Fixations"] = final_df["Fixations"].astype("Int64")
final_df["Inview"] = final_df["Inview"].astype("Int64")
final_df

,hostname,Impressions,Fixations,Fixation Ratio,Inview,Inview Ratio,Inview / Fixation Ratio,Sample Size
0,affarsvarlden.se,2800,0,0.00,1168,41.71,0.00,high
1,aftonbladet.se,965536,235794,24.42,322212,33.37,73.18,high
2,allabolag.se,18667,0,0.00,3327,17.82,0.00,high
3,archive.md,2400,0,0.00,2400,100.00,0.00,high
4,azaleasdolls.com,1,0,0.00,1,100.00,0.00,low
...,...,...,...,...,...,...,...,...
75,wowhead.com,487,0,0.00,7,1.44,0.00,high
76,wrestlezone.com,443,0,0.00,0,0.00,0.00,high
77,wrestlinginc.com,2783,0,0.00,2484,89.26,0.00,high
78,zelda.fandom.com,1,0,0.00,0,0.00,0.00,low


In [19]:
final_df.to_excel(f"../final_reports/OBOS_campaign_xaxis_{group}.xlsx", index=False)